# تمرین سری هفتم
object detection on **Pascal VOC** using **Fast R-CNN**

## هدف
 یادگیری استفاده از الگوریتم Fast R-CNN برای وظایف شناسایی اشیا.


# در پایان این تمرین، شما قادر خواهید بود یک مدل Fast R-CNN آموزش دهید و از آن برای شناسایی اشیا در تصاویر استفاده کنید.

------------------------------------------------------

# بخش اول


راه اندازی محیط

In [1]:
# نصب کتابخانه‌های مورد نیاز
!pip install tensorflow opencv-python matplotlib

In [2]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as ET



# بخش دوم



آماده سازی داده

دانلود مجموعه داده
**Pascal VOC**  
و آماده سازی آن.

 آماده‌سازی مجموعه داده با تبدیل حاشیه‌نویسی‌ها به فرمت مورد نیاز (جعبه‌های مرزی و برچسب‌های کلاس).

In [ ]:
def load_pascal_voc_dataset(data_path):
    """
    بارگذاری مجموعه داده Pascal VOC و آماده‌سازی آن.

    ورودی‌ها:
    - data_path (str): مسیر دایرکتوری که مجموعه داده در آن قرار دارد.

    خروجی‌ها:
    - images (list of str): لیست مسیرهای تصاویر.
    - bboxes (list of list): لیست جعبه‌های مرزی برای هر تصویر.
    - labels (list of list): لیست برچسب‌ها برای هر تصویر.
    """
    images = []
    bboxes = []
    labels = []
    for root, _, files in os.walk(data_path):
        for file in files:
            if file.endswith('.xml'):
                tree = ET.parse(os.path.join(root, file))
                root = tree.getroot()
                filename = root.find('filename').text
                image_path = os.path.join(root.find('folder').text, filename)
                images.append(image_path)
                boxes = []
                lbls = []
                for obj in root.findall('object'):
                    bbox = obj.find('bndbox')
                    box = [
                        int(bbox.find('xmin').text),
                        int(bbox.find('ymin').text),
                        int(bbox.find('xmax').text),
                        int(bbox.find('ymax').text)
                    ]
                    boxes.append(box)
                    lbls.append(obj.find('name').text)
                bboxes.append(boxes)
                labels.append(lbls)
    return images, bboxes, labels

data_path = '/path/to/VOCdevkit/VOC2012'  # مسیر خود را به مجموعه داده Pascal VOC تنظیم کنید
images, bboxes, labels = load_pascal_voc_dataset(data_path)



# بخش سوم


بارگزاری و نمایش مجموعه داده ها

In [ ]:
#TODO: نوشتن تابع برای نمایش تصاویر و جعبه‌های مرزی آن‌ها
def visualize_image_with_bboxes(image_path, bboxes, labels=None):
    """
    نمایش یک تصویر با جعبه‌های مرزی.

    ورودی‌ها:
    - image_path (str): مسیر فایل تصویر.
    - bboxes (list of list): لیست جعبه‌های مرزی، هرکدام به صورت [xmin, ymin, xmax, ymax].
    - labels (list of str, اختیاری): لیست برچسب‌ها که متناظر با جعبه‌های مرزی هستند.

    خروجی‌ها:
    - هیچکدام: تصویر با جعبه‌های مرزی نمایش داده می‌شود.
    """
    image = cv2.imread(image_path)
    for i, bbox in enumerate(bboxes):
        cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 255, 0), 2)
        if labels:
            cv2.putText(image, labels[i], (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# نمایش نمونه‌ای از داده‌ها
visualize_image_with_bboxes(images[0], bboxes[0], labels[0])



# مرحله 4


آماده سازی مدل

بارگذاری یک مدل پیش‌آموزش دیده

***Fast R-CNN***

از مدل

***zoo***

در


***tensorflow***

In [3]:
# مرحله 4: آماده‌سازی مدل
#  از مدل زو TensorFlow (Fast R-CNN می‌تواند به روشی مشابه استفاده شود).

# TODO: نوشتن تابع برای بارگذاری یک مدل Faster R-CNN پیش‌آموزش دیده
import tensorflow_hub as hub
def load_pretrained_model():
    """
    بارگذاری یک مدل Faster R-CNN پیش‌آموزش دیده از مدل زو TensorFlow.

    خروجی‌ها:
    - model: مدل Faster R-CNN پیش‌آموزش دیده.
    """
    # دانشجویان باید این قسمت را تکمیل کنند
    # URL to a pre-trained Faster R-CNN model in TensorFlow Hub
    model_url = "https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1"

    # Load the pre-trained model
    model = hub.load(model_url)
    return model

model = load_pretrained_model()

# مرحله ششم

استنتاج

نوشتن تابع برای انجام استنتاج بر روی یک تصویر با استفاده از مدل


***Faster R-CNN.***

In [4]:
# TODO: نوشتن تابع برای پیش‌پردازش تصویر برای استنتاج مدل
def preprocess_image(image_path):
    """
    پیش‌پردازش یک تصویر برای استنتاج مدل.

    ورودی‌ها:
    - image_path (str): مسیر فایل تصویر.

    خروجی‌ها:
    - input_image (آرایه numpy): تصویر پیش‌پردازش شده آماده برای ورودی مدل.
    - original_image (آرایه numpy): تصویر اصلی برای نمایش.
    """
    # دانشجویان باید این قسمت را تکمیل کنند
    # Read the image
    original_image = cv2.imread(image_path)
    if original_image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")

    # Convert image to RGB (from BGR, which is default in OpenCV)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    # Resize the image to the size expected by the model (640x640 for the pre-trained Faster R-CNN model)
    input_image = cv2.resize(original_image, (640, 640))

    # Normalize the image: the pre-trained Faster R-CNN model expects pixel values in the range [0, 1]
    input_image = input_image.astype(np.float32) / 255.0

    return input_image, original_image

In [ ]:
# TODO: نوشتن تابع برای شناسایی اشیا
def detect_objects(model, image_path, threshold=0.5):
    """
    شناسایی اشیا در یک تصویر با استفاده از یک مدل پیش‌آموزش دیده.

    ورودی‌ها:
    - model: مدل پیش‌آموزش دیده.
    - image_path (str): مسیر فایل تصویر.
    - threshold (شناور): آستانه شناسایی.

    خروجی‌ها:
    - هیچکدام: تصویر با جعبه‌های مرزی شناسایی شده نمایش داده می‌شود.
    """
    # دانشجویان باید این قسمت را تکمیل کنند
    # Preprocess the image
    input_image, original_image = preprocess_image(image_path)

    # Convert the preprocessed image to a tensor and add a batch dimension
    input_tensor = tf.convert_to_tensor(input_image)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Perform inference
    detections = model(input_tensor)

    # Extract detection results
    detection_boxes = detections['detection_boxes'][0].numpy()
    detection_scores = detections['detection_scores'][0].numpy()
    detection_classes = detections['detection_classes'][0].numpy().astype(int)

    # Get the height and width of the original image
    height, width, _ = original_image.shape

    # Draw bounding boxes and labels on the original image
    for i in range(len(detection_boxes)):
        if detection_scores[i] >= threshold:
            box = detection_boxes[i] * [height, width, height, width]
            box = box.astype(int)
            class_id = detection_classes[i]
            score = detection_scores[i]

            # Draw bounding box
            cv2.rectangle(original_image, (box[1], box[0]), (box[3], box[2]), (0, 255, 0), 2)

            # Draw label and score
            label = f"Class {class_id}: {score:.2f}"
            cv2.putText(original_image, label, (box[1], box[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the image with bounding boxes
    plt.imshow(original_image)
    plt.axis('off')
    plt.show()

# دانشجویان باید 'path_to_your_image.jpg' را با یک مسیر تصویر واقعی جایگزین کنند
detect_objects(model, images[0])

# مرحله هفتم

ارزیابی

In [5]:
def compute_iou(box1, box2):
    ymin1, xmin1, ymax1, xmax1 = box1
    ymin2, xmin2, ymax2, xmax2 = box2

    inter_ymin = max(ymin1, ymin2)
    inter_xmin = max(xmin1, xmin2)
    inter_ymax = min(ymax1, ymax2)
    inter_xmax = min(xmax1, xmax2)

    inter_area = max(0, inter_ymax - inter_ymin) * max(0, inter_xmax - inter_xmin)
    box1_area = (ymax1 - ymin1) * (xmax1 - xmin1)
    box2_area = (ymax2 - ymin2) * (xmax2 - xmin2)
    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area if union_area != 0 else 0
    return iou

In [6]:
def compute_map(detections, annotations, iou_threshold):
    average_precisions = []
    for class_id in set([cls for det in detections for cls in det[2]]):
        true_positives = []
        scores = []
        num_annotations = 0
        for i in range(len(detections)):
            detected_boxes, detected_scores, detected_classes = detections[i]
            gt_boxes, gt_labels = annotations[i]
            gt_boxes = [box for j, box in enumerate(gt_boxes) if gt_labels[j] == class_id]
            detected_boxes = [box for j, box in enumerate(detected_boxes) if detected_classes[j] == class_id]
            detected_scores = [score for j, score in enumerate(detected_scores) if detected_classes[j] == class_id]
            num_annotations += len(gt_boxes)

            if len(gt_boxes) == 0:
                true_positives.extend([0] * len(detected_boxes))
                scores.extend(detected_scores)
                continue

            detected_boxes = np.array(detected_boxes)
            gt_boxes = np.array(gt_boxes)
            scores.extend(detected_scores)

            for d, detected_box in enumerate(detected_boxes):
                ious = [compute_iou(detected_box, gt_box) for gt_box in gt_boxes]
                max_iou = max(ious) if ious else 0
                if max_iou >= iou_threshold:
                    true_positives.append(1)
                else:
                    true_positives.append(0)

        if num_annotations == 0:
            average_precisions.append(0)
            continue

        sorted_indices = np.argsort(-np.array(scores))
        true_positives = np.array(true_positives)[sorted_indices]
        false_positives = 1 - true_positives

        cum_true_positives = np.cumsum(true_positives)
        cum_false_positives = np.cumsum(false_positives)
        precision = cum_true_positives / (cum_true_positives + cum_false_positives)
        recall = cum_true_positives / num_annotations

        precision = np.concatenate([[0], precision, [0]])
        recall = np.concatenate([[0], recall, [1]])

        for i in range(len(precision) - 1, 0, -1):
            precision[i - 1] = np.maximum(precision[i - 1], precision[i])

        indices = np.where(recall[1:] != recall[:-1])[0]
        average_precision = np.sum((recall[indices + 1] - recall[indices]) * precision[indices + 1])
        average_precisions.append(average_precision)

    mAP = np.mean(average_precisions)
    return mAP

In [7]:
# TODO:  نوشتن تابع ارزیابی مدل با استفاده از معیار های خواسته شده
def evaluate_model(model, dataset, iou_threshold=0.5):
    """
    ارزیابی عملکرد مدل بر روی یک مجموعه اعتبارسنجی.

    ورودی‌ها:
    - model: مدل آموزش دیده.
    - dataset: مجموعه داده اعتبارسنجی.

    خروجی‌ها:
    - metrics: دیکشنری حاوی معیارهای ارزیابی مانند میانگین دقت متوسط (mAP).
    """
    # دانشجویان باید این قسمت را تکمیل کنند
    all_detections = []
    all_annotations = []
    for image_path, ground_truth_boxes, ground_truth_labels in tqdm(dataset):
        input_image, _ = preprocess_image(image_path)
        input_tensor = tf.convert_to_tensor(input_image)
        input_tensor = input_tensor[tf.newaxis, ...]

        detections = model(input_tensor)

        detection_boxes = detections['detection_boxes'][0].numpy()
        detection_scores = detections['detection_scores'][0].numpy()
        detection_classes = detections['detection_classes'][0].numpy().astype(int)

        filtered_boxes = []
        filtered_scores = []
        filtered_classes = []
        for i in range(len(detection_scores)):
            if detection_scores[i] >= iou_threshold:
                filtered_boxes.append(detection_boxes[i])
                filtered_scores.append(detection_scores[i])
                filtered_classes.append(detection_classes[i])

        all_detections.append((filtered_boxes, filtered_scores, filtered_classes))
        all_annotations.append((ground_truth_boxes, ground_truth_labels))

    mAP = compute_map(all_detections, all_annotations, iou_threshold)
    return {'mAP': mAP}